In [178]:
import pandas as pd
import os
from datetime import datetime
from dateutil import parser
import re
import matplotlib.pyplot as plt

In [32]:
# Declare relevant paths
curr_dir = os.getcwd()
scraped_vaccine_dir = os.path.join(curr_dir, 'Scraped Vaccine')
scraped_covid_dir = os.path.join(curr_dir, 'Scraped COVID')

In [40]:
# First we need some way of parsing the dates from the file names.
# Take, for example, the file names of the vaccine data.
# We can only infer the date from the ending of the file name.
os.listdir(scraped_vaccine_dir)

['Data Vaksinasi Berbasis Kelurahan (03 September 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (28 Juli 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (15 Juli 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (25 Agustus 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (06 September 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (31 Juli 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (05 Agustus 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (04 Oktober 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (08 Agustus 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (25 Juni 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (03 Juli 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (09 September 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (27 Juli 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (18 Agustus 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (15 Agustus 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan (16 Agustus 2021).xlsx',
 'Data Vaksinasi Berbasis Kelurahan 

In [ ]:
# To parse the date, we would need the help of dateutil.parser to transform the string to a datetime object.
# However, notice that the date string is in Indonesian (specifically the name of the months).
# We will have to manually translate them into english so we can parse them (the library doesn't recognize Indonesian months).

In [45]:
# Dictionary that maps Indonesian months into English months
month_dict = {'Januari':'January', 
              'Februari':'February', 
              'Maret':'March', 
              'April':'April', 
              'Mei':'May', 
              'Juni':'June', 
              'Juli':'July', 
              'Agustus':'August', 
              'September':'September', 
              'Oktober':'October',
              'November':'November',
              'Desember':'December',
             }

In [112]:
def infer_date(file_name, file_type):
    # Obtain date from the ending of the file_name
    if file_type=='Vaccine':
        date_in_filename = str(file_name[34:])
    elif file_type=='COVID':
        date_in_filename = str(file_name[30:])
    else:
        pass
    
    # Erase the brackets, then the .xlsx extension
    date_indonesian = re.sub('[()]+', '', date_in_filename)
    date_indonesian = re.sub('.xlsx', '', date_indonesian).strip()
    
    # Split by space and get the month (middle word)
    indonesian_month = date_indonesian.split(' ')[1]
    english_month = month_dict.get(indonesian_month, 'Not in Dict')
    
    # Replace Indonesian month with English month
    date_english = [word for i,word in enumerate(date_indonesian.split(' ')) if i<=2] # keep only 3 first words (date, month, year)
    date_english[1] = english_month
    
    # Join back the words into one string
    date_english = ' '.join(date_english)
    
    # Now that we have the date in English, parse it into datetime and add it as new column in the dataframe.
    datetime_object = parser.parse(date_english)
    
    return datetime_object

In [123]:
# Demo
text_vaccine = os.listdir(scraped_vaccine_dir)[0]
text_covid = os.listdir(scraped_covid_dir)[0]

print('ORIGINAL')
print(f'  Vaccine file name: {text_vaccine}')
print(f'  COVID file name: {text_covid}')

print('PARSED')
print(f"  Vaccine date: {infer_date(text_vaccine, 'Vaccine')}")
print(f"  COVID date: {infer_date(text_covid, 'COVID')}")

ORIGINAL
  Vaccine file name: Data Vaksinasi Berbasis Kelurahan (03 September 2021).xlsx
  COVID file name: Standar Kelurahan Data Corona (04 Mei 2021 Pukul 10.00).xlsx
PARSED
  Vaccine date: 2021-09-03 00:00:00
  COVID date: 2021-05-04 00:00:00


In [115]:
# Function taking Excel file path as input, giving date-appended dataframe and date (string) as output
def consume_excel(dir_path, file_name, file_type):
    
    full_path = os.path.join(dir_path, file_name)
    
    df = pd.read_excel(full_path, 
                       engine = 'openpyxl',
                       header = [0],
                       sheet_name=0)
    
    # Throw away first row
    df = df.iloc[1:,:]
    
    # Parse ending of file_name into datetime object
    datetime_object = infer_date(file_name, file_type)
    
    # Now that we have the date in English, parse it into datetime and add it as new column in the dataframe.
    df['date'] = datetime_object
    
    return df, datetime.strftime(datetime_object, '%Y-%m-%d')
    

### Populate Vaccine Data

In [116]:
vaccine_dict = {}

# Populate vaccine_dict with vaccine dataframes
for vaccine_file in os.listdir(scraped_vaccine_dir):
    # ensure to read vaccine files
    if 'Data Vaksinasi' in vaccine_file:
        df, date = consume_excel(scraped_vaccine_dir, vaccine_file, 'Vaccine')
        vaccine_dict[date] = df   

### Populate COVID Data

In [121]:
covid_dict = {}

# Populate covid_dict with COVID dataframes
for covid_file in os.listdir(scraped_covid_dir):
    # ensure to read vaccine files
    if 'Data Corona' in covid_file:
        df, date = consume_excel(scraped_covid_dir, covid_file, 'COVID')
        covid_dict[date] = df   

### Stack each dataframe into one (one for COVID and one for Vaccine)

In [131]:
covid_data = pd.DataFrame()

for key in covid_dict.keys():
    covid_data = pd.concat([covid_data, covid_dict[key]], axis=0)

In [132]:
vaccine_data = pd.DataFrame()

for key in vaccine_dict.keys():
    vaccine_data = pd.concat([vaccine_data, vaccine_dict[key]], axis=0)

In [146]:
# Clean column names
vaccine_cols = [re.sub('[\n]+',' ',col).lower() for col in vaccine_data.columns]
covid_cols = [re.sub('[\n]+',' ',col).lower() for col in covid_data.columns]

vaccine_data.columns = vaccine_cols
covid_data.columns = covid_cols

### Preview Data

In [148]:
pd.set_option('display.max_columns', None)

In [196]:
vaccine_data.head()

,kode kelurahan,wilayah kota,kecamatan,kelurahan,sasaran,belum vaksin,jumlah dosis 1,jumlah dosis 2,total vaksin diberikan,lansia dosis 1,lansia dosis 2,lansia total vaksin diberikan,pelayan publik dosis 1,pelayan publik dosis 2,pelayan publik total vaksin diberikan,gotong royong dosis 1,gotong royong dosis 2,gotong royong total vaksin diberikan,tenaga kesehatan dosis 1,tenaga kesehatan dosis 2,tenaga kesehatan total vaksin diberikan,tahapan 3 dosis 1,tahapan 3 dosis 2,tahapan 3 total vaksin diberikan,remaja dosis 1,remaja dosis 2,remaja total vaksin diberikan,date
1,3.172051e+09,JAKARTA UTARA,PADEMANGAN,ANCOL,23947,9067,14880,9678,24558,1302,1147,2449,4201,3269,7470,89,82,171,107,96,203,7481.0,3765.0,11246.0,1700.0,1319.0,3019.0,2021-09-03
2,3.173041e+09,JAKARTA BARAT,TAMBORA,ANGKE,29381,10275,19106,11138,30244,2108,1869,3977,3649,2629,6278,137,120,257,92,85,177,11124.0,4973.0,16097.0,1996.0,1462.0,3458.0,2021-09-03
3,3.175041e+09,JAKARTA TIMUR,KRAMAT JATI,BALE KAMBANG,29074,12345,16729,8480,25209,1056,789,1845,4301,2663,6964,188,165,353,229,204,433,8838.0,3380.0,12218.0,2117.0,1279.0,3396.0,2021-09-03
4,3.175031e+09,JAKARTA TIMUR,JATINEGARA,BALI MESTER,9752,3664,6088,3894,9982,917,825,1742,1371,1030,2401,80,71,151,78,74,152,3038.0,1481.0,4519.0,604.0,413.0,1017.0,2021-09-03
5,3.175101e+09,JAKARTA TIMUR,CIPAYUNG,BAMBU APUS,26285,9218,17067,11250,28317,1443,1272,2715,4574,3505,8079,146,114,260,380,345,725,8342.0,4341.0,12683.0,2182.0,1673.0,3855.0,2021-09-03
